<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import sklearn
import numpy as np
import pandas as pd

In [130]:
from google.colab import drive
drive.mount('/content/drive')
wildfire_set1 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/wildfires1_w_snow.csv')
wildfire_set2 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/wildfires2_w_snow.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Now that we have gathered data from kaggle, NOAA, and the California Department of Water resources into two consistent datasets, we will drop columns that are not necessary for training our models and take care of some final preprocessing.

In this file, I will 

In [131]:
print(wildfire_set1.columns)

Index(['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Fire Size Rank', 'Cause',
       'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month',
       'Discovered DOY', 'Contained Month', 'Contained DOY', 'Latitude',
       'Longitude', 'County', 'CountyIds', 'State', 'OWNER_DESCR',
       'NOAA Station', 'Link', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP', 'TAVG',
       'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Snow Station', 'River Basin', 'Mar_SP', 'Mar_WC',
       'Mar_Dens'],
      dtype='object')


In [132]:
print(wildfire_set2.columns)

Index(['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Fire Size Rank', 'Cause',
       'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month',
       'Discovered DOY', 'Contained Month', 'Contained DOY', 'Latitude',
       'Longitude', 'County', 'CountyIds', 'State', 'OWNER_DESCR',
       'NOAA Station', 'Link', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP', 'TAVG',
       'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Snow Station', 'River Basin', 'Mar_SP', 'Mar_WC',
       'Mar_Dens'],
      dtype='object')


Drop all columns that will not be relevant for our classification task:
Name, AcresBurned, Cause, SOURCE_REPORTING UNIT NAME, DaysBurn, Discovered DOY, Contained Month, Contained DOY, Lat, Long, County Ids, State, OWNER_DESCR, NOAA Station, Link, Snow Station, River Basin

In [133]:
wildfire_set1.drop(columns = ['Unnamed: 0', 'Year', 'Name', 'Cause',
                      'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month', 'Contained Month',
                      'Contained DOY','Latitude','Longitude','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin'], inplace = True)

In [134]:
wildfire_set2.drop(columns = ['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Cause',
                      'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month', 'Contained Month',
                      'Contained DOY','Latitude','Longitude','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin'], inplace = True)

Double check our datatypes before we proceed with preprocessing and model building. As we can see, all features besides county and fire size rank are numerical (of type float) so all we have to do is one hot encode these two categorial 

In [135]:
wildfire_set1.head(5)

,AcresBurned,Fire Size Rank,Discovered DOY,County,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens
0,0.10,A,33.0,Plumas,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,49.06,0.0,19.0,1.0,79.6,34.0,0.43
1,0.25,A,133.0,Placer,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,14.76,0.0,3.0,1.0,108.6,38.1,0.35
2,0.10,A,152.0,El Dorado,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,40.37,0.0,11.0,1.0,108.6,38.1,0.35
3,0.10,A,180.0,Alpine,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33
4,0.10,A,180.0,Alpine,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33


In [136]:
wildfire_set2.head(5)

,Fire Size Rank,Discovered DOY,County,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens
0,G,NaN,Monterey,6.5,0.0,0.0,0.0,0.00,58.8,65.2,52.4,14.11,21.42,0.0,1.0,0,0.0,0.0,0.00
1,G,NaN,Kern,6.7,529.0,0.0,22.0,0.00,82.6,96.6,68.6,4.68,4.88,15.0,4.0,1,36.0,16.0,0.44
2,G,NaN,San Luis Obispo,6.9,237.0,0.0,23.0,0.00,72.6,92.6,52.6,2.52,8.09,43.0,0.0,0,0.0,0.0,0.00
3,G,NaN,San Bernardino,6.5,455.0,0.0,28.0,0.00,79.7,94.6,64.7,3.41,6.45,43.0,0.0,0,0.0,0.0,0.00
4,G,NaN,Siskiyou,4.5,0.0,0.0,0.0,0.02,56.4,62.9,49.9,18.03,54.17,0.0,2.0,1,77.0,34.0,0.44


In [137]:
print(wildfire_set1['Fire Size Rank'].value_counts())

A    59783
B    46740
C     5470
D     1187
E      654
F      448
G      276
Name: Fire Size Rank, dtype: int64


In [138]:
print(wildfire_set2['Fire Size Rank'].value_counts())

C    601
D    238
E    119
F    114
G     97
A     23
B      5
Name: Fire Size Rank, dtype: int64


In [139]:
w = wildfires_larger.loc[wildfires_larger['AcresBurned'] >= 3]
print(w['Fire Size Rank'].value_counts())

B    3540
C    2684
D     657
E     432
F     327
G     234
Name: Fire Size Rank, dtype: int64


In [140]:
w['New Fire Size Rank'] = ''
for index, rows in w.iterrows():
  size_fire = w.at[index, 'AcresBurned']
  if size_fire >= 3 and size_fire <= 5:
    size_class = 'A'
  elif size_fire > 5 and size_fire <= 40:
    size_class = 'B'
  elif size_fire > 40:
    size_class = 'C'
  w.at[index,'New Fire Size Rank'] = size_class

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [141]:
print(w['New Fire Size Rank'].value_counts())

B    3069
A    2466
C    2339
Name: New Fire Size Rank, dtype: int64


In [142]:
wildfires_larger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61572 entries, 0 to 61571
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AcresBurned         61572 non-null  float64
 1   Fire Size Rank      61572 non-null  object 
 2   Discovered DOY      61572 non-null  float64
 3   County              61528 non-null  object 
 4   AWND                58465 non-null  float64
 5   CLDD                59549 non-null  float64
 6   DP10                59636 non-null  float64
 7   DX90                59549 non-null  float64
 8   PRCP                59636 non-null  float64
 9   TAVG                59549 non-null  float64
 10  TMAX                59549 non-null  float64
 11  TMIN                59587 non-null  float64
 12  PRCP_6M             60437 non-null  float64
 13  PRCP_RS             60127 non-null  float64
 14  DX90_2M             60667 non-null  float64
 15  DP10_2M             60925 non-null  float64
 16  Rece

In [143]:
wildfires_larger.describe()

,AcresBurned,Discovered DOY,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens
count,61572.000000,61572.000000,58465.000000,59549.000000,59636.000000,59549.000000,59636.000000,59549.000000,59549.000000,59587.000000,60437.000000,60127.000000,60667.000000,60925.000000,61475.000000,61475.000000,61475.000000,61475.000000
mean,129.327407,199.661275,6.243207,230.615090,0.993226,12.068666,0.490340,69.025846,82.730761,55.309834,9.827607,17.513501,18.036000,3.058022,0.528101,28.198455,11.370339,0.193609
std,3265.111650,67.168299,1.897651,229.509513,1.868922,11.926164,1.185171,11.622404,13.941725,10.420073,10.335150,14.981989,20.414305,4.425545,0.499214,39.410314,16.562603,0.219441
min,0.010000,1.000000,0.400000,0.000000,0.000000,0.000000,0.000000,18.300000,33.000000,3.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.100000,163.000000,4.900000,7.000000,0.000000,0.000000,0.000000,59.600000,72.100000,48.500000,2.390000,6.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.100000,202.000000,5.800000,175.000000,0.000000,8.000000,0.070000,70.200000,84.600000,56.000000,6.260000,12.160000,8.000000,2.000000,1.000000,0.000000,0.000000,0.000000
75%,1.000000,243.000000,7.400000,398.000000,1.000000,24.000000,0.430000,78.200000,94.400000,62.700000,14.010000,26.020000,35.000000,4.000000,1.000000,54.200000,20.800000,0.390000
max,315578.800000,366.000000,17.200000,1113.000000,24.000000,31.000000,20.590000,100.900000,114.300000,89.000000,85.910000,88.290000,62.000000,38.000000,1.000000,215.000000,113.000000,1.560000


In [144]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

Address Null Values and Data Smoothing

One-hot encode categorical features (e.g. Month discovered)

Bar Charts and Class Definition

Split the data into train-test sets

In [145]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

Normalize data using min-max-scalar after splitting into train and test

Feature Selection

In [146]:
from sklearn.feature_selection import RFE, SelectKBest

Cross-Validation

Model Building:

SVM Classifier

In [147]:
from sklearn import svm
from sklearn.svm import SVC

KNN Classifier

In [148]:
from sklearn.neighbors import KNeighborsClassifier

Naive Bayes Classifier

In [149]:
from sklearn.naive_bayes import GaussianNB

Decision Tree Classifier

In [150]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

Random Forest Classifier

In [151]:
from sklearn.ensemble import RandomForestClassifier

Gradient Boost Classifier

In [152]:
from sklearn.ensemble import GradientBoostingClassifier

Neural Network Multiclass classifier (TensorFlow)
- Experiment by changing number of hidden layers and activation functions (sigmoid, relu, softmax)
- Change number of epochs and add more hidden layers
- Size of input = number of features in the dataset
- Size of output = number of classes in the multiclass classification problem

In [153]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns

Voting Classifier

In [154]:
from sklearn.ensemble import VotingClassifier